<a href="https://colab.research.google.com/github/lphohmann/DL_microbial_gene_classifier/blob/main/build_datablock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
# do i need this still? 
#!pip install fastai --upgrade

In [1]:
#hide
!pip install torchtext==0.8.1
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 7.0MB 4.7MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 727kB 5.2MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 1.2MB 8.5MB/s 
     |████████████████████████████████| 204kB 19.4MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
Mounted at /content/gdrive


In [2]:
#hide
import fastai
from fastbook import *

In [3]:
import numpy as np

In [4]:
# mount google drive to access files and set the correct working
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/ColabNotebooks/DL_project/
!pwd

Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks/DL_project
/content/drive/MyDrive/ColabNotebooks/DL_project


In [ ]:
# create a path object to the files, not sure if still needed
path = Path('/content/drive/MyDrive/ColabNotebooks/DL_project')
# View all files in directory
path.ls()

In [6]:
# cell to check function documentation
??DataBlock

In [5]:
# read in my data from which training and validation set will be created
trainval = pd.read_csv('trainval.csv', low_memory=False)

In [6]:
# defining functions required for building the DataBlock
# get_x and y by specifying column in dataframe
def get_x(r): return r['Seq']
def get_y(r): return r['Knum']
# item_tfms: transform functions to one-hot encode and to pad to same size
# one hot encoding function
from sklearn.preprocessing import OneHotEncoder
def OH_enc(seq: str):
    cats = {'K', 'D', 'N', 'E', 'R', 'A', 'T', 'L', 'I', 'Q', 'C', 'F', 'G', 'W', 'M', 'S', 'H', 'P', 'V', 'Y'}
    cat_array = np.array(cats, ndmin=1)
    # get seq into array
    seq_array = np.array(list(seq))
    #one hot encode the sequence
    onehot_encoder = OneHotEncoder(categories=[cat_array],sparse=False,handle_unknown='ignore')
    onehot_encoded_seq = onehot_encoder.fit_transform(seq_array.reshape(len(seq_array), 1))
    #print(onehot_encoder.categories_)
    return np.transpose(onehot_encoded_seq)

# zero padding function
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0):
    pad_size = target_length - array.shape[axis]
    if pad_size <= 0:
        return array
    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)
    return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

# main item_tfms functino
def main_item_tfms(sample_touple):
    seq = sample_touple[0]
    enc_seq = OH_enc(seq)
    pad_encseq_array = pad_along_axis(enc_seq,2820,1)
    return pad_encseq_array,sample_touple[1]


In [ ]:
# build the datablock
dblock = DataBlock(
                 splitter = TrainTestSplitter(test_size=0.2, random_state=42, stratify=trainval[['Knum']]),
                 get_x = get_x,
                 get_y = get_y,
                 item_tfms=main_item_tfms) # check that it is applied to the sequence not the label, look into documentation 
dblock.summary(trainval)                    # change the trasnfrom function so it takes a touple as input and only applies to the first item 


In [9]:
# create dataloaders from datablock
dls = dblock.dataloaders(trainval) # doesnt work yet?